In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.express as px
import sys, datetime
sys.path.append('/jupyter/ext')
import stackql
%load_ext stackql

In [ ]:
owner = widgets.Text(
    description='GitHub Owner',
    disabled=False
)

repo = widgets.Text(
    description='GitHub Repo',
    disabled=False
)

sprint_number = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Sprint Number',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    disabled=False
)

display(sprint_number)
display(start_date)
display(end_date)
display(owner)
display(repo)

In [ ]:

def show_contributions(df):
    fig = px.bar(df,
             x='login',
             y='contributions',
             title='Contributions by User',
             labels={'login': 'User Login', 'contributions': 'Number of Contributions'},
             text='contributions')

    fig.update_layout(xaxis_tickangle=-45)

    fig.show()